In [6]:
import numpy as np
import pandas as pd
import re

In [7]:
dfs = {tck: pd.read_csv(f"../data/{tck}.csv") for tck in ("AAPL", "DELL", "IBM")}

# Process Dell data
def process_dell_prices(col):
    return [float(re.findall("(?<=\$).*", x)[0]) for x in col]

dfs["DELL"]["Close"] = process_dell_prices(dfs["DELL"]["Close/Last"])
dfs["DELL"]["Open"] = process_dell_prices(dfs["DELL"]["Open"])
dfs["DELL"]["High"] = process_dell_prices(dfs["DELL"]["High"])
dfs["DELL"]["Low"] = process_dell_prices(dfs["DELL"]["Low"])

In [18]:
def discretize(x, n_points):
    edges = np.linspace(x.min(), x.max(), num=n_points)
    return np.sum(x.reshape(-1, 1) > edges, axis=1)

def get_3d_1d_obs(df):
    frac_change = np.array((df.Close - df.Open) / df.Open)
    frac_high = np.array((df.High - df.Open) / df.Open)
    frac_low = np.array((df.Open - df.Low) / df.Open)

    x = discretize(frac_change, n_points=51)
    y = discretize(frac_high, n_points=11)
    z = discretize(frac_low, n_points=11)

    n = (z - 1) * (x.max() * y.max()) + (y - 1) * x.max() + x

    return n, (x, y, z)

In [19]:
xyzs = {}
ns = {}
for tck in dfs.keys():
    n, xyz = get_3d_1d_obs(dfs[tck])
    xyzs[tck] = xyz
    ns[tck] = n